# SageMaker 에서 한국어 모델 LoRA 파인튜닝

---

## 이 노트북은 SageMaker Studio Notebook PyTorch 1.13 Python 3.9 docker image 와 ml.t3.medium 에서 테스트 되었습니다. :
### 참조 자료:
- 파인튜닝 fine-tuning 😎 내 데이터로 학습해보자 - 스탠포드 알파카 Stanford Alpaca 코드분석 (feat, LLaMa + GPT3.5 vs ChatGPT)
    - https://www.youtube.com/watch?v=u2tQYgrLouo
- stanford_alpaca
    - https://github.com/tatsu-lab/stanford...
- alpaca-lora
    - https://github.com/tloen/alpaca-lora

- Quick intro: PEFT or Parameter Efficient Fine-tuning
    - [PEFT](https://github.com/huggingface/peft), or Parameter Efficient Fine-tuning, is a new open-source library from Hugging Face to enable efficient adaptation of pre-trained language models (PLMs) to various downstream applications without fine-tuning all the model's parameters. PEFT currently includes techniques for:
- LoRA: [LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS](https://arxiv.org/pdf/2106.09685.pdf)
- Prefix Tuning: [P-Tuning v2: Prompt Tuning Can Be Comparable to Fine-tuning Universally Across Scales and Tasks](https://arxiv.org/pdf/2110.07602.pdf)
- P-Tuning: [GPT Understands, Too](https://arxiv.org/pdf/2103.10385.pdf)
- Prompt Tuning: [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/pdf/2104.08691.pdf)


# 1. 기본 환경 설정

In [1]:
# is_first_time = True
is_first_time = False
if is_first_time:
    !pip install "transformers==4.26.0" "datasets[s3]==2.9.0" sagemaker py7zr --upgrade --quiet

In [3]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('../../common_code')

If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.



In [4]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)


print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")
print(f"bucket: {sagemaker_session_bucket}")


sagemaker role arn: arn:aws:iam::057716757052:role/gen_ai_gsmoon
sagemaker bucket: sagemaker-us-east-1-057716757052
sagemaker session region: us-east-1
bucket: sagemaker-us-east-1-057716757052


# 2. LLM 모델 지정

In [5]:
model_id = "nlpai-lab/kullm-polyglot-12.8b-v2"
model_name = model_id.split('/')[1]
model_name = model_name.split('.')[0]
print("model_name: ", model_name)

model_name:  kullm-polyglot-12


# 3. 데이터 세트 로딩


In [6]:
from scripts.KoAlpacaData import KoAlpacaData

dataset_name = 'ko-alpaca-data'
download_folder = "../../Data"

# KoAlpacaDataset = KoAlpacaData(download_folder)
KoAlpacaDataset = KoAlpacaData(download_folder, split_rate = 0.99, is_download=False)

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Existing data is used
data_file_path:  ../../Data/ko_alpaca_data.json
train_end:  49123


In [7]:
def sample_dataset(dataset, sample_size, sample_json_file):
    sample_json_file = f"{download_folder}/ko_alpaca_data_train_sample.json"
    dataset = KoAlpacaDataset.sample_dataset(dataset, sample_size, train_sample_json_file)
    return dataset


In [8]:

train_dataset = KoAlpacaDataset.train_dataset
len(train_dataset)

sample_size = 100    
train_sample_json_file = f"{download_folder}/ko_alpaca_data_train_sample.json"
sample_train_dataset = sample_dataset(train_dataset, sample_size, train_sample_json_file)

Using custom data configuration default-6dacbb2d1f9be5fe


Size of sample dataset:  49685


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 65.52it/s]


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-6dacbb2d1f9be5fe/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


# 4. 훈련 데이터 셋 준비

In [9]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.model_max_length = 2048 # overwrite wrong value

### 데이터 세트 템플릿 정의

In [10]:
import os
os.system("mkdir -p templates")

0

In [11]:
import json

prompt_template = f"다읨의 질문에 답변을 하세요:\n{{question}}\n---\n답변:\n{{answer}}{{eos_token}}"
prompt_template

'다읨의 질문에 답변을 하세요:\n{question}\n---\n답변:\n{answer}{eos_token}'

### 데이터 세트를 템플릿 형태로 생성

In [17]:
from random import randint
from itertools import chain
from functools import partial


# custom instruct prompt start
# prompt_template = f"Summarize the chat dialogue:\n{{dialogue}}\n---\nSummary:\n{{summary}}{{eos_token}}"

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = prompt_template.format(question=sample["instruction"],
                                            answer=sample["output"],
                                            eos_token=tokenizer.eos_token)
    return sample

prompt_dataset = sample_train_dataset.map(template_dataset, 
                                          remove_columns=list(sample_train_dataset.features))
# prompt_dataset = prompt_dataset['train']

100%|██████████| 100/100 [00:00<00:00, 4154.59ex/s]


In [18]:
print(prompt_dataset[randint(0, len(prompt_dataset))]["text"])

다읨의 질문에 답변을 하세요:
물의 끓는점을 찾아보세요.
---
답변:
물의 끓는점은 약 100도입니다. 그러나 물의 끓는점은 해발 고도와 같은 외부 요인에 의해 변할 수 있습니다.<|endoftext|>


### 데이터 세트를 Transformer 입력 형태로 인코딩

In [19]:
# apply prompt template per sample

# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": []}


def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


In [20]:
# tokenize and chunk dataset
lm_dataset = prompt_dataset.map(
    lambda sample: tokenizer(sample["text"], return_token_type_ids=False), batched=True, remove_columns=list(prompt_dataset.features)
).map(
    partial(chunk, chunk_length=1536),
    batched=True,
)
lm_dataset

100%|██████████| 1/1 [00:00<00:00, 29.23ba/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4
})

### 인코딩 데이터 세트를 S3 에 업로딩

In [21]:
bucket_prefix = f'LLM/instructon_data/KoAlpaca/train'
training_input_path = f's3://{sagemaker_session_bucket}/{bucket_prefix}'
print(f"training dataset to: \n {training_input_path}")

training dataset to: 
 s3://sagemaker-us-east-1-057716757052/LLM/instructon_data/KoAlpaca/train


In [22]:
# save train_dataset to s3

lm_dataset.save_to_disk(training_input_path)

print(f"uploaded data to: {training_input_path}")


uploaded data to: s3://sagemaker-us-east-1-057716757052/LLM/instructon_data/KoAlpaca/train


# 5. LoRA and bnb int-8 파인 튜닝



In [23]:
import time
# define Training Job Name 
job_name = f'{model_name}-{dataset_name}-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
print("training job_name: ", job_name)

from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                                # pre-trained model
  'dataset_path': '/opt/ml/input/data/training', # path where sagemaker will save training dataset
  'epochs': 1,                                         # number of training epochs
  'per_device_train_batch_size': 1,                    # batch size for training
  'lr': 2e-4,                                          # learning rate used during training
}

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_clm.py',      # train script
    source_dir           = 'scripts',         # directory which includes all the files needed for training
    instance_type        = 'ml.g5.2xlarge', # instances type used for the training job
#    instance_type        = 'ml.p4d.24xlarge', # instances type used for the training job    
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    hyperparameters      =  hyperparameters
)

training job_name:  kullm-polyglot-12-ko-alpaca-data-2023-08-04-00-27-13


In [24]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=False)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: kullm-polyglot-12-ko-alpaca-data-2023-0-2023-08-04-00-27-15-901


Using provided s3_resource


In [25]:
training_job_name = huggingface_estimator.latest_training_job.job_name
print("training_job_name: \n", training_job_name)

training_job_name: 
 kullm-polyglot-12-ko-alpaca-data-2023-0-2023-08-04-00-27-15-901


In [26]:
huggingface_estimator.logs()

2023-08-04 00:27:16 Starting - Starting the training job...
2023-08-04 00:27:32 Starting - Preparing the instances for training......
2023-08-04 00:28:36 Downloading - Downloading input data...
2023-08-04 00:28:56 Training - Downloading the training image..................
2023-08-04 00:32:13 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-08-04 00:32:45,196 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-08-04 00:32:45,210 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-08-04 00:32:45,220 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-08-04 00:32:45,222 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-08-04 00:32:45,424 sagemaker-training-toolkit INFO     Installing dependencie

# 6. SageMaker Endpoint 배포



기존의 Estimator 를 통해서 model_data (모델 가중치가 저장된 S3 위치) 구할 수 있지만, 추후에 사용을 위해서 훈련 잡 이름을 가지고 다시 Estimaotr 를 생성하여 배포를 하였습니다.

## 훈련 잡 이름으로 Estimator 다시 생성

In [27]:
training_job_name = 'kullm-polyglot-12-ko-alpaca-data-2023-0-2023-07-11-12-33-03-618'

In [28]:


from sagemaker.estimator import Estimator
attached_estimator = Estimator.attach(training_job_name)



2023-07-11 13:17:14 Starting - Preparing the instances for training
2023-07-11 13:17:14 Downloading - Downloading input data
2023-07-11 13:17:14 Training - Training image download completed. Training in progress.
2023-07-11 13:17:14 Uploading - Uploading generated training model
2023-07-11 13:17:14 Completed - Training job completed


## SageMaker Model 을 생성하여 모델 배포

In [29]:
from sagemaker.huggingface import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=attached_estimator.model_data,
   #model_data="s3://hf-sagemaker-inference/model.tar.gz",  # Change to your model path
   role=role, 
   transformers_version="4.26", 
   pytorch_version="1.13", 
   py_version="py39",
   model_server_workers=1
)

We can now deploy our model using the `deploy()` on our HuggingFace estimator object, passing in our desired number of instances and instance type.

In [30]:
from datetime import datetime

time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
endpoint_name = f"{model_name}-" + time_stamp

In [ ]:
%%time

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   endpoint_name= endpoint_name,    
   initial_instance_count=1,
   instance_type= "ml.g5.4xlarge"
)

INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2023-08-04-01-12-05-836
INFO:sagemaker:Creating endpoint-config with name kullm-polyglot-12-2023-08-04-01-12-05
INFO:sagemaker:Creating endpoint with name kullm-polyglot-12-2023-08-04-01-12-05


--

In [ ]:
endpoint_name

# 7. 모델 추론

endpoint_name 은 아해 하드 코딩이 된 것은 무시하시고, 위에서 생성한 것을 사용하세요.

In [ ]:
endpoint_name = 'kullm-polyglot-12-2023-07-15-02-49-17'

In [ ]:
def create_prompt(prompt):

    fomatted_sample = {
      # "inputs": prompter.generate_prompt(q, c),
      "inputs": prompt,  
      "parameters": {
        "do_sample":False, 
        "max_new_tokens":256,
        "temperature":0.5,
        "top_k":3,
        "top_p":0.9,
        "repetition_penalty":1.1,
        "presence_penalty":None,
        "eos_token_id":2,              
      }
    }
    print("fomatted_sample: \n", fomatted_sample)
    
    return fomatted_sample

def generate_llm(prompt):
    # predict
    res = predictor.predict(prompt)

    print(res[0]["generated_text"].split("Summary:")[-1])


    

## 맥락이 있는 것 을 질문

In [160]:
from inference_lib import invoke_inference_DJ,  parse_response, Prompter

prompter = Prompter("kullm")

In [167]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = "홈플러스 영업시간은 오전 10시 부터 오후 12시까지 입니다."


# prompt_w_c = f"아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.\n 요청을 적절히 완료하는 답변을 작성하세요.\n### 명령어: {q}\n\n### 입력: {c}\n\n### 답변:" if c else f"### 명령어: {q}\n\n### 답변:" 
prompt_w_c = prompter.generate_prompt(q, c)

prompt_w_c

'아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.\n 요청을 적절히 완료하는 답변을 작성하세요.\n ### 명령어:\n홈플러스 중계점은 몇시까지 장사해?\n\n### 입력:\n홈플러스 영업시간은 오전 10시 부터 오후 12시까지 입니다.\n\n### 답변:\n'

In [168]:
fomatted_prompt = create_prompt(prompt_w_c)

fomatted_sample: 
 {'inputs': '아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.\n 요청을 적절히 완료하는 답변을 작성하세요.\n ### 명령어:\n홈플러스 중계점은 몇시까지 장사해?\n\n### 입력:\n홈플러스 영업시간은 오전 10시 부터 오후 12시까지 입니다.\n\n### 답변:\n', 'parameters': {'do_sample': False, 'max_new_tokens': 256, 'temperature': 0.5, 'top_k': 3, 'top_p': 0.9, 'repetition_penalty': 1.1, 'presence_penalty': None, 'eos_token_id': 2}}


In [169]:
res = invoke_inference_DJ(endpoint_name, fomatted_prompt)
parse_response(res)

'아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.\n 요청을 적절히 완료하는 답변을 작성하세요.\n ### 명령어:\n홈플러스 중계점은 몇시까지 장사해?\n\n### 입력:\n홈플러스 영업시간은 오전 10시 부터 오후 12시까지 입니다.\n\n### 답변:\n홈플러스 중계점의 영업 시간은 오전 10시부터 오후 12시까지입니다.'

## 맥락이 없는 것을 질문

In [179]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = "홈플러스 중계점은 노원구에 있습니다."


prompt_w_c = f"아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.\n 요청을 적절히 완료하는 답변을 작성하세요.\n\
맥락에서 답변을 얻지 못하면 정보가 없다고 하세요.\n\
### 명령어: {q}\n\n### 입력: {c}\n\n### 답변:" if c else f"### 명령어: {q}\n\n### 답변:" 
# prompt_w_c = prompter.generate_prompt(q, c)

prompt_w_c

'아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.\n 요청을 적절히 완료하는 답변을 작성하세요.\n맥락에서 답변을 얻지 못하면 정보가 없다고 하세요.\n### 명령어: 홈플러스 중계점은 몇시까지 장사해?\n\n### 입력: 홈플러스 중계점은 노원구에 있습니다.\n\n### 답변:'

In [178]:
fomatted_prompt = create_prompt(prompt_w_c)
res = invoke_inference_DJ(endpoint_name, fomatted_prompt)
parse_response(res)

fomatted_sample: 
 {'inputs': '아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.\n 요청을 적절히 완료하는 답변을 작성하세요.\n맥락에서 답변을 얻지 못하면 정보가 없다고 하세요.\n### 명령어: 홈플러스 중계점은 몇시까지 장사해?\n\n### 입력: 홈플러스 중계점은 노원구에 있습니다.\n\n### 답변:', 'parameters': {'do_sample': False, 'max_new_tokens': 256, 'temperature': 0.5, 'top_k': 3, 'top_p': 0.9, 'repetition_penalty': 1.1, 'presence_penalty': None, 'eos_token_id': 2}}


'아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.\n 요청을 적절히 완료하는 답변을 작성하세요.\n맥락에서 답변을 얻지 못하면 정보가 없다고 하세요.\n### 명령어: 홈플러스 중계점은 몇시까지 장사해?\n\n### 입력: 홈플러스 중계점은 노원구에 있습니다.\n\n### 답변:\n죄송합니다만, 저는 AI 언어 모델이기 때문에 실시간 위치 데이터나 날씨 업데이트에 액세스할 수 없습니다. 제 답변은 인터넷의 정보를 기반으로 한 것이며, 지역 및 시간에 따라 달라질 수 있으므로 정확한 답변을 드릴 수 없습니다.'

## 테스트 데이터 셋으로 추론

In [166]:
test_dataset = KoAlpacaDataset.test_dataset
len(test_dataset)
test_dataset[0:5]


[{'instruction': '주어진 단어의 순위를 매기고 가장 강한 단어부터 가장 약한 단어까지 순서대로 나열합니다.',
  'input': '들어올리기, 당기기, 밀기',
  'output': '들어올리기, 밀기, 당기기 순으로 단어의 순위를 매긴다면 들어올리기가 가장 강한 단어이며, 밀기가 가장 약한 단어가 됩니다.'},
 {'instruction': '주어진 입력에 따라 사람을 설명하는 형용사 두 개를 생성하세요.',
  'input': '기발한',
  'output': '기발한 사람은 창의적이면서도 흥미로운 특성을 지니고 있습니다.'},
 {'instruction': '주어진 지침에 따라 범죄 유형을 분류하세요.',
  'input': '피고가 구내에 불법적으로 침입했습니다.',
  'output': '해당 사건은 침입죄에 해당됩니다.'},
 {'instruction': '다양한 데이터 저장 방법을 설명하세요.',
  'input': '',
  'output': '다양한 데이터 저장 방법에는 하드 디스크, USB 드라이브, 클라우드 서버, 데이터베이스 등이 있습니다.'},
 {'instruction': '해질녘 도시의 한 장면을 묘사하십시오.',
  'input': '',
  'output': '도시는 적막하고, 일부 건물에서는 주말 불이 켜져있습니다. 몇몇 차들이 도로를 지나가며 시내 소음을 내고 있습니다.'}]

In [170]:

q = "주어진 지침에 따라 범죄 유형을 분류하세요"    
c = "피고가 구내에 불법적으로 침입했습니다."
prompt_w_c = prompter.generate_prompt(q, c)
fomatted_prompt = create_prompt(prompt_w_c)

invoke_inference_DJ(endpoint_name, fomatted_prompt)

fomatted_sample: 
 {'inputs': '아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.\n 요청을 적절히 완료하는 답변을 작성하세요.\n ### 명령어:\n주어진 지침에 따라 범죄 유형을 분류하세요\n\n### 입력:\n피고가 구내에 불법적으로 침입했습니다.\n\n### 답변:\n', 'parameters': {'do_sample': False, 'max_new_tokens': 256, 'temperature': 0.5, 'top_k': 3, 'top_p': 0.9, 'repetition_penalty': 1.1, 'presence_penalty': None, 'eos_token_id': 2}}


'[{"generated_text":"아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다.\\n 요청을 적절히 완료하는 답변을 작성하세요.\\n ### 명령어:\\n주어진 지침에 따라 범죄 유형을 분류하세요\\n\\n### 입력:\\n피고가 구내에 불법적으로 침입했습니다.\\n\\n### 답변:\\n불법 침입은 형법의 범죄 유형 중 하나입니다."}]'

# 8. 엔드포인트 삭제

In [97]:
# predictor.delete_model()
# predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: huggingface-pytorch-inference-2023-07-11-13-59-10-188
INFO:sagemaker:Deleting endpoint configuration with name: kullm-polyglot-12-2023-07-11-13-58-58
INFO:sagemaker:Deleting endpoint with name: kullm-polyglot-12-2023-07-11-13-58-58


# A. Reference
- LLM 모델에 필요한 template 파일을 정의하는 코드
    - https://dacon.io/en/codeshare/8449
```
# GPT, LLama, Alpaca 등 LLM 모델에 필요한 template 파일을 정의하는 코드입니다.
# 아래 코드는 Alpaca-Lora가 인식하는 template의 형식입니다. 아래처럼 진행해서, 마찬가지로 json 파일로 저장해주시면 됩니다.

prompt_template = {
    "description" : "Alpaca-Lora Custom Template",
    "prompt_input" : (
        "Below is an instruction that describes a task, Paired with an input that provides further context.\n"
        "Writhe a response that appropriately completes the request.\n"
        "### Instruction : \n{instruction}\n\n### Input : \n{input}\n\n### Response:\n"
    ),
    "prompt_no_input" : (
        "Below is an instruction that describes a task.\n"
        "Writhe a response that appropriately completes the request.\n"
        "### Instruction : \n{instruction}\n\n### Input : \n{input}\n\n### Response:\n"
    ),
    "response_split" : "### Response:"
}
```